### The City of Scottsdale publishes a [dataset](http://data.scottsdaleaz.gov/dataset/planning-and-development-cases) that includes planning and development cases dating back to the 1970s.

### How many hearings, and which type of hearings does each case go through as part of the case hearing process? 

In [2]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# databaseConnection = 'DRIVER={SQL Server}; SERVER=ServerName; Database=DatbaseName; UID=UserId; PWD=password;'
connection = pyodbc.connect('DRIVER={SQL Server};SERVER=WINDOWS-A6S01I2\SQLEXPRESS;DATABASE=City_of_Scottsdale;Trusted_Connection=yes;')
cursor = connection.cursor()

# query excludes observations that don't have 'meeting type', 'meeting type' is ambiguous or
# the meeting took place prior to when Scottsdale was incorporated(1951), probably human data entry error
Case_Query = '''
select *
from [dbo].[pds_CaseMeetings]
where [MeetingType] <> '' and [MeetingType] <> 'Other' and year([MeetingDate]) > '1951' 
  '''
# save the sql query to a pandas dataframe
Case_Data = pd.read_sql(Case_Query, connection)
connection.close()

Case_Data['MeetingDate'] = pd.to_datetime(Case_Data['MeetingDate']) #converte MeetingDate to datetime format
Case_Data['MeetingDate'] = Case_Data['MeetingDate'].dt.year #converte MeetingDate to year

Case_Data.head()

,meeting_id,case_id,CaseNumber,MeetingType,MeetingDate,Result,Vote
0,7571,20987,4-UP-2001,City Council Hearing,2001,APPROVED,7-0
1,7573,20988,5-UP-2001,Planning Commission Hearing,2001,CONTINUED,
2,7574,20988,5-UP-2001,City Council Hearing,2001,APPROVED,7-0
3,7575,20990,3-ZN-2001,Planning Commission Hearing,2001,CONTINUED,6-0
4,7576,20990,3-ZN-2001,City Council Hearing,2001,APPROVED,7-0


In [3]:
Case_Data.case_id.count()

15401

In [5]:
# it seems both case_id and CaseNumber are unique and roughly similar
Case_Data.CaseNumber.count()

15401

In [6]:
Case_Data.meeting_id.count()

15401

In [7]:
Case_Data.meeting_id.value_counts().head()

54667    1
32510    1
24862    1
5691     1
33691    1
Name: meeting_id, dtype: int64

In [20]:
Case_Data.CaseNumber.value_counts().head(10)

                1773
12-GP-1999        22
6-TA-2009#2       21
19-ZN-1987#2      14
12-GP-1994        14
65-ZN-1992#7      14
2-ZN-2000         14
3-TA-2000         13
2-II-2010         13
9-UP-2000         12
Name: CaseNumber, dtype: int64

In [21]:
Case_Data.groupby(Case_Data.CaseNumber).get_group('9-UP-2000').sort_values(by='MeetingDate', ascending=False)

,meeting_id,case_id,CaseNumber,MeetingType,MeetingDate,Result,Vote
367,10866,19935,9-UP-2000,City Council Hearing,2002,APPROVED,7-0
9602,10397,19935,9-UP-2000,City Council Hearing,2002,CONTINUED,
10710,10488,19935,9-UP-2000,City Council Hearing,2002,CONTINUED,
11051,10783,19935,9-UP-2000,City Council Hearing,2002,CONTINUED,
8277,7540,19935,9-UP-2000,City Council Hearing,2001,,
8278,7541,19935,9-UP-2000,City Council Hearing,2001,ON HOLD,
9403,9476,19935,9-UP-2000,City Council Hearing,2001,CONTINUED,
9429,9747,19935,9-UP-2000,City Council Hearing,2001,CONTINUED,
9547,10213,19935,9-UP-2000,City Council Hearing,2001,CONTINUED,7-0
10192,8833,19935,9-UP-2000,City Council Hearing,2001,CONTINUED,


In [8]:
Case_Data.head()

,meeting_id,case_id,CaseNumber,MeetingType,MeetingDate,Result,Vote
0,7571,20987,4-UP-2001,City Council Hearing,2001,APPROVED,7-0
1,7573,20988,5-UP-2001,Planning Commission Hearing,2001,CONTINUED,
2,7574,20988,5-UP-2001,City Council Hearing,2001,APPROVED,7-0
3,7575,20990,3-ZN-2001,Planning Commission Hearing,2001,CONTINUED,6-0
4,7576,20990,3-ZN-2001,City Council Hearing,2001,APPROVED,7-0


In [18]:

Case_Data.groupby([Case_Data.case_id, Case_Data.MeetingType]).size().reset_index().sort_values(by=[0, "case_id"], ascending=False)

,case_id,MeetingType,0
13282,757,City Council Hearing,11
5076,2696,City Council Hearing,11
2393,1865,City Council Hearing,11
2394,1865,Planning Commission Hearing,11
4756,25973,Development Review Hearing,10
2807,19935,City Council Hearing,10
12288,562,City Council Hearing,9
8232,38117,Planning Commission Hearing,9
2906,20351,Development Review Hearing,9
9352,41582,Development Review Hearing,8
